## Connect with the DB



In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import os
import re

# Database credentials (from user's previous code)
DB_USER = "user_invest"
DB_PASSWORD = "senha_forte"
DB_HOST = 'localhost'
DB_PORT = '5432'
DB_NAME = 'postgres'
TABLE_NAME = 'companys'

try:
    # Create the database connection URL
    db_url = f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
    engine = create_engine(db_url)

    # List of tickers to analyze, including Ibovespa
    tickers_list = ["PETR4.SA", "ELET3.SA", "EGIE3.SA", "CPFE3.SA", "CMIG4.SA", "ENEV3.SA",
                    "JBSS3.SA", "BRFS3.SA", "MRFG3.SA", "BEEF3.SA", "SMTO3.SA", "ABEV3.SA",
                    "CCRO3.SA", "ECOR3.SA", "SBSP3.SA", "CSMG3.SA", "WEGE3.SA", "^BVSP"]
    
    # Query to fetch the data
    query = f"""
    SELECT data, ticker, "Fechamento" 
    FROM {TABLE_NAME} 
    WHERE ticker IN ({', '.join([f"'{t}'" for t in tickers_list])}) 
    ORDER BY data;
    """

    # Read data from the database into a pandas DataFrame
    df = pd.read_sql(query, engine)

df['ticker'] = df['ticker'].apply(lambda x: re.sub(r'(\.SA)$', '', x).replace('^', ''))
    
    # Reorganiza o DataFrame para que cada ticker seja uma coluna
    df_pivot = df.pivot_table(index='data', columns='ticker', values='Fechamento')

    # Normaliza os dados para que todos comecem em 100%
    df_normalized = df_pivot.div(df_pivot.iloc[0]) * 100

    # Cria o gráfico de linhas
    plt.style.use('seaborn-v0_8-whitegrid')
    plt.figure(figsize=(15, 10))

    # Plota cada linha no gráfico
    for column in df_normalized.columns:
        plt.plot(df_normalized.index, df_normalized[column], label=column)

    plt.title('Performance Normalizada de Ações e Ibovespa ao Longo do Tempo', fontsize=18)
    plt.xlabel('Data', fontsize=14)
    plt.ylabel('Preço Normalizado (%)', fontsize=14)
    plt.legend(title='Ticker', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True)
    plt.tight_layout(rect=[0, 0, 0.85, 1])

    # Salva o gráfico em um arquivo
    plot_filename = 'performance_normalizada.png'
    plt.savefig(plot_filename)
    print(f"Gráfico de performance normalizada salvo como '{plot_filename}'")
    
except Exception as e:
    print(f"Ocorreu um erro: {e}")

Conexão bem-sucedida! Dados carregados:
         data ticker  Abertura   Maxima   Minima  Fechamento   Volume
0  2015-01-05  ^BVSP   48512.0  48512.0  47264.0     47517.0  3866100
1  2015-01-06  ^BVSP   47517.0  48061.0  47338.0     48001.0  4559300
2  2015-01-07  ^BVSP   48006.0  49882.0  48006.0     49463.0  4408800
3  2015-01-08  ^BVSP   49463.0  50261.0  49017.0     49943.0  3621900
4  2015-01-09  ^BVSP   49955.0  49955.0  48501.0     48840.0  2999200

Informações do DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2477 entries, 0 to 2476
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   data        2477 non-null   object 
 1   ticker      2477 non-null   object 
 2   Abertura    2477 non-null   float64
 3   Maxima      2477 non-null   float64
 4   Minima      2477 non-null   float64
 5   Fechamento  2477 non-null   float64
 6   Volume      2477 non-null   int64  
dtypes: float64(4), int64(1), object(2